# Analise de Sentimento

### Baixando o Córpus

Existe um córpus livre de avaliações disponibilizados pela empresa B2W/americanas.com,
que pode ser baixado a partir do seguinte 
[endereço do github](https://github.com/b2wdigital/b2w-reviews01)

O córpus é disponibilizado em dois formatos, Ou com um  banco de dados sqlite3  ou como uma planilha no formato csv (comma separated values),  que nada mais é do que uma lista separada por vírgulas de todos os itens,   uma entrada por linha.  

Neste tutorial iremos usar o formato csv.  
A planilha possui 132.374  entradas e foi disponibilizada ao público em 2019.   
Para esse exercício sugerimos uma versão reduzida deste corpus com 10 mil entradas,
no arquivo `data/b2w-10k.csv`. 

Vamos experimentar com este arquivo.


In [0]:
!python -m spacy download pt

     |████████████████████████████████| 12.9MB 620kB/s 
  Created wheel for pt-core-news-sm: filename=pt_core_news_sm-2.1.0-cp36-none-any.whl size=12843677 sha256=c0a066094e8910e4dc7ca9836df9fa4f13f211f52bdd5d5f7960e28d2e2ea2ef
  Stored in directory: /tmp/pip-ephem-wheel-cache-g6kb5iul/wheels/a3/8f/c1/f036e3a7f1aa44fb06a534c6c4b1c2b773f101fdb1f163c08c
Successfully built pt-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/pt_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/pt
You can now load the model via spacy.load('pt')


In [0]:
!mkdir -p data

In [0]:
!wget -c https://github.com/advinstai/dsml-school2019/raw/master/day4/data/b2w-10k.csv -O data/b2w-10k.csv

--2019-12-19 19:01:56--  https://github.com/advinstai/dsml-school2019/raw/master/day4/data/b2w-10k.csv
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/advinstai/dsml-school2019/master/day4/data/b2w-10k.csv [following]
--2019-12-19 19:01:57--  https://raw.githubusercontent.com/advinstai/dsml-school2019/master/day4/data/b2w-10k.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4013451 (3.8M) [text/plain]
Saving to: ‘data/b2w-10k.csv’

data/b2w-10k.csv    100%[===================>]   3.83M  --.-KB/s    in 0.05s   

2019-12-19 19:01:57 (79.7 MB/s) - ‘data/b2w-10k.csv’ saved [4013451/4013451]



In [0]:
import pandas as pd
import csv

b2wCorpus = pd.read_csv("data/b2w-10k.csv")
b2wCorpus.head()

,submission_date,reviewer_id,product_id,product_name,product_brand,site_category_lv1,site_category_lv2,review_title,overall_rating,recommend_to_a_friend,review_text,reviewer_birth_year,reviewer_gender,reviewer_state,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,2018-01-01 00:11:28,d0fb1ca69422530334178f5c8624aa7a99da47907c44de...,132532965,Notebook Asus Vivobook Max X541NA-GO472T Intel...,NaN,Informática,Notebook,Bom,4,Yes,Estou contente com a compra entrega rápida o ú...,1958,F,RJ,NaN,NaN,NaN,NaN,NaN
1,2018-01-01 00:13:48,014d6dc5a10aed1ff1e6f349fb2b059a2d3de511c7538a...,22562178,Copo Acrílico Com Canudo 500ml Rocie,NaN,Utilidades Domésticas,"Copos, Taças e Canecas","Preço imbatível, ótima qualidade",4,Yes,"Por apenas R$1994.20,eu consegui comprar esse ...",1996,M,SC,NaN,NaN,NaN,NaN,NaN
2,2018-01-01 00:26:02,44f2c8edd93471926fff601274b8b2b5c4824e386ae4f2...,113022329,Panela de Pressão Elétrica Philips Walita Dail...,philips walita,Eletroportáteis,Panela Elétrica,ATENDE TODAS AS EXPECTATIVA.,4,Yes,SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...,1984,M,SP,NaN,NaN,NaN,NaN,NaN
3,2018-01-01 00:35:54,ce741665c1764ab2d77539e18d0e4f66dde6213c9f0863...,113851581,Betoneira Columbus - Roma Brinquedos,roma jensen,Brinquedos,Veículos de Brinquedo,presente mais que desejado,4,Yes,MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS D...,1985,F,SP,NaN,NaN,NaN,NaN,NaN
4,2018-01-01 01:00:28,7d7b6b18dda804a897359276cef0ca252f9932bf4b5c8e...,131788803,"Smart TV LED 43"" LG 43UJ6525 Ultra HD 4K com C...",lg,TV e Home Theater,TV,"Sem duvidas, excelente",5,Yes,"A entrega foi no prazo, as americanas estão de...",1994,M,MG,NaN,NaN,NaN,NaN,NaN


O córpus tem uma série de informações,  mas para esse treinamento iremos usar apenas 
duas. A coluna de entrada será `review_text`, contendo o texto da avaliação em 
português brasileiro,  e a coluna de saída será `recommend_to_a_friend`  contendo a
recomendação de indicar o produto para um amigo;  a coluna de saída tem apenas dois 
valores: "Yes" ou "No".

Claramente o arquivo foi preparado para ser usado pela Comunidade Internacional e não apenas para os que falam português.  Vamos preparar uma versão com apenas estas colunas e armazená-la na planilha `data/b2w-10k-short.csv`

In [0]:
binary= [1 if str == "Yes" else 0 for str in b2wCorpus["recommend_to_a_friend"]]
textos = list(b2wCorpus["review_text"])

data = {'Texto': textos, 'Julgamento':binary}
df = pd.DataFrame(data) 
df.to_csv("data/b2w-10k-short.csv",index=False)

df.head()

,Texto,Julgamento
0,Estou contente com a compra entrega rápida o ú...,1
1,"Por apenas R$1994.20,eu consegui comprar esse ...",1
2,SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...,1
3,MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS D...,1
4,"A entrega foi no prazo, as americanas estão de...",1


Preparação dos dados

Vamos sparar o córpus em 3 partes: treino, validação e teste.

In [0]:
import numpy  as np
import pandas as pd
import csv
MAIN_CSV  = "data/b2w-10k-short.csv"
VAL_RATIO = 0.1
TRAIN_CSV = "data/b2w-train-10k-short.csv"
VAL_CSV   = "data/b2w-val-10k-short.csv"
TEST_CSV  = "data/b2w-test-10k-short.csv"

df = pd.read_csv(MAIN_CSV)

# Enbaralha as linhas da entrada
idx = np.arange(df.shape[0])
np.random.seed()
np.random.shuffle(idx)

# train: 80%, val: 10%, test: 10%
valSize = int(len(idx) * VAL_RATIO)
df.iloc[idx[2*valSize:],        0:2].to_csv(TRAIN_CSV,index=False, quoting=csv.QUOTE_NONNUMERIC)
df.iloc[idx[:valSize],          0:2].to_csv(VAL_CSV, index=False, quoting=csv.QUOTE_NONNUMERIC)
df.iloc[idx[valSize:2*valSize], 0:2].to_csv(TEST_CSV, index=False, quoting=csv.QUOTE_NONNUMERIC)

Para tokenizar, usamos expressões regulares do pacote `re` em conjunção com o tokenizador do `spacy`.
E para gerar os datasets, usamos a biblioteca `torchtext` e suas classes `Field` e `TabularDataset`.

In [0]:
import torch
import spacy
import re
from torchtext import data
from torchtext import datasets

from os import path

torch.manual_seed(1234)

spacy_pt = spacy.load('pt')
def tokenizePT(s):
    comment = re.sub(r"[\*\"“”\n\\…\+\-\/\=\(\)‘•:\[\]\|’\!;]", " ", str(s))
    comment = re.sub(r"[ ]+", " ", comment)
    comment = re.sub(r"\!+", "!", comment)
    comment = re.sub(r"\,+", ",", comment)
    comment = re.sub(r"\.+", ".", comment)
    comment = re.sub(r"\?+", "?", comment)
    return [tok.text for tok in spacy_pt.tokenizer(comment) if tok.text != " "]

UNK = "<unk>"
PAD = "<pad>"    
SOS = "<s>"
EOS = "</s>"
LOWER = True
MIN_FREQ = 20

TEXT  = data.Field(tokenize=tokenizePT, batch_first=True, lower=LOWER, 
                 include_lengths=True, unk_token=UNK, pad_token=PAD, 
                 init_token=SOS, eos_token=EOS)
LABEL = data.LabelField(dtype=torch.float,use_vocab=False, sequential=False)

trainDataset, valDataset, testDataset = data.TabularDataset.splits(
            path       = path.dirname( TRAIN_CSV ),
            train      = path.basename(TRAIN_CSV), 
            validation = path.basename(VAL_CSV), 
            test       = path.basename(TEST_CSV),
            format     = 'csv', 
            skip_header= True,
            fields     = [('Texto', TEXT),('Julgamento', LABEL)]
)

TEXT.build_vocab(trainDataset, valDataset, testDataset, min_freq=MIN_FREQ) 
print("Size vocab: ", len(TEXT.vocab))    
print("Size datasetS: ", len(trainDataset), len(valDataset), len(testDataset))    
 
for i in range(10):
    print(TEXT.vocab.itos[i])

Size vocab:  1135
Size datasetS:  8001 999 999
<unk>
<pad>
<s>
</s>
.
,
o
e
a
de


In [0]:
for t in trainDataset.Texto:
    print(t)
    break
for j in trainDataset.Julgamento:
    print(j)
    break
for t in valDataset.Texto:
    print(t)
    break
for t in testDataset.Texto:
    print(t)
    break


['comprei', 'o', 'console', 'xbox', '360', '4', 'gb', 'controle', 'sem', 'fio', ',', 'faturado', 'em', 'novembro', ',', 'e', 'o', 'jogo', 'comprei', 'depois', 'e', 'chegou', 'primeiro', ',', 'e', 'ele', 'até', 'agora', 'nada,.e', 'ai', 'está', 'esperando', 'o', 'que', '?']
0
['voltarei', 'a', 'comprar', 'nas', 'americanas', '.indico', '.', 'sempre', 'veio', 'conforme', 'o', 'pedido', '.', 'rápido', 'na', 'entrega.nota', '10']
['pedi', ',', 'paguei', ',', 'venceu', 'o', 'prazo', 'da', 'entrega', 'e', 'ainda', 'não', 'recebi', 'o', 'produto', '.', 'na', 'hora', 'da', 'compra', 'informaram', '9', 'dias', 'úteis', ',', 'quando', 'finalizei', 'já', 'passou', 'para', '10', 'dias', 'úteis', '.', 'passou', 'o', 'prazo', 'e', 'só', 'a', 'nota', 'foi', 'emitida', ',', 'nem', 'foi', 'para', 'transporte', 'ainda', '.', 'vou', 'devolver']


Agora vamos criar iteradores que irão dividir ir os nossos corpus em lotes do 
tamanho `BATCH_ SIZE`

In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [0]:
BATCH_SIZE = 8

trainIter, validIter, testIter = data.BucketIterator.splits(
    (trainDataset, valDataset, testDataset), 
    batch_size=BATCH_SIZE, 
    sort_key=lambda x: len(x.Texto),
    sort_within_batch=True,
    repeat=False,
    device=device)

for b in trainIter:
    print(b.Texto, b.Julgamento)
    break
for b in validIter:
    print(b.Texto, b.Julgamento)
    break
for b in testIter:
    print(b.Texto, b.Julgamento)
    break

(tensor([[   2,   48,  155,    5,  169,    5,   77,  470,    5,  129,  510,  291,
            4,    3],
        [   2,  658,    9,    0,    4,    0,   14,    0,    5,   11,    0,    0,
            4,    3],
        [   2,   33,   10,   22,    0,    4,   21,   74,    7,   16,   19,   68,
            4,    3],
        [   2,   32,   23,  110,  252,    5,   79,    0,  852,  138,  123,  630,
            4,    3],
        [   2,   10,   32,   38,   15,  161,    7,   50,    6,  184,   72,  354,
            4,    3],
        [   2,  658,    9,    0,    0,  611,    4,   32,   38,   15,   31,  426,
            4,    3],
        [   2,   10,    9,   70,   25,    7,   16,   21,   96,   15,   31,    4,
           20,    3],
        [   2,    0,   14,  459,    9,   10,    9,   25, 1048,    5,   13,   19,
          115,    3]], device='cuda:0'), tensor([14, 14, 14, 14, 14, 14, 14, 14], device='cuda:0')) tensor([1., 1., 1., 1., 1., 1., 1., 1.], device='cuda:0')
(tensor([[  2,  79,  26, 111,   3],
   

## Construição do modelo

Utilizamos uma arquitetura RNN denominada LSTM (Long Short-Term Memory). Nela o estado oculto pode ser pensado como uma "memória" das palavras vistas pelo modelo. As redes LSTMs têm um estado recorrente extra chamado _cell_, que pode ser considerado a "memória" do LSTM e pode lembrar informações por várias etapas. As LSTMs também usam vários _gates_, controlam o fluxo de informações para dentro e para fora da memória. Para mais informações, acesse [aqui] (https://colah.github.io/posts/2015-08-Understanding-LSTMs/).

### LSTM bidirecional

O conceito por trás de uma RNN bidirecional é simples. Além de ter uma RNN direta processando as palavras na frase da primeiro à última posição, temos uma segunda RNN reversa processando as palavras na frase da **última à primeira**. Numa LSTM bidirecional o estado oculto final retornado, `hidden`, é a concatenação do estado oculto da última palavra da frase da RNN direta com o estado oculto da primeira palavra da sentença da RNN reversa, sendo ambos os estados ocultos finais de suas respectivas RNNs.

![](https://i.imgur.com/itmIIgx.png)

### RNN de várias camadas

RNN multicamada (também chamados de *RNN profunda*) é outro conceito simples. A idéia é que adicionemos RNNs adicionais sobre a RNN padrão inicial, onde cada RNN adicionada é outra *camada*. A saída do estado oculto pela primeira RNN (inferior) no tempo $t$ será a entrada para a RNN acima dela no tempo $t$. A previsão é geralmente feita a partir do estado oculto final da camada final (mais alta). Estes são facilmente combinados com RNNs bidirecionais, onde cada camada extra adiciona um RNN adicional direta e reversa.

![](https://i.imgur.com/knsIzeh.png)

### Regularização

Quanto mais parâmetros você tiver em seu modelo, maior a probabilidade de que você se ajuste demais (tenha um erro baixo de treino, mas alto erro de validação/teste, chamado de _overfitting_). Para combater isso, usamos de regularização. Mais especificamente, usamos um método de regularização chamado *dropout*, o qual funciona aleatoriamente *retirando* (ddando peso 0 a) neurônios durante uma passagem da rede. A probabilidade de que cada neurônio seja eliminado é definida por um hiperparâmetro e cada neurônio com é considerado para o _dropout_ de forma independente. Um modelo com parâmetros eliminados pode ser visto como um modelo "mais fraco" (menos parâmetros); as previsões de todos esses modelos "mais fracos" (um para cada passagem da rede) são calculadas em conjunto nos parâmetros do modelo. Assim, seu único modelo pode ser pensado como um conjunto de modelos mais fracos, nenhum dos quais é super parametrizado e, portanto, não deve estar super ajustado.

### Detalhes da implementação

Para usar uma LSTM usamos `nn.LSTM`. Observe também que a LSTM retorna um `outpit` e uma tupla do estado final `hidden` e o estado final da célula de memória.

Como o estado oculto final da LSTM possui um componente direto e reverso, que são concatenados, o tamanho da entrada para a camada `nn.Linear` é o dobro do tamanho da dimensão oculta.

A implementação da bidirecionalidade e a adição de camadas adicionais são feitas passando valores para os argumentos `num_layers` e` bidirectional` para o RNN/LSTM. Note que estamos usando um embedding padrão através da classe `nn.Embedding`, que recebe o tamanho do vocábulário e o tamanho do vetor de embedding como parâmetros.

A regularização dropout é implementada inicializando uma camada `nn.Dropout` (o argumento é a probabilidade de dropout para cada neurônio) e usando-o dentro do método `forward` após cada camada na qual queremos aplicar o dropout. **Nota**: nunca use dropout nas camadas de entrada ou saída (`x` ou` fc` neste caso), você só deseja usar dropout em camadas intermediárias. A LSTM possui um argumento `dropout` que adiciona dropout nas conexões entre estados ocultos em uma camada e estados ocultos da próxima camada.

In [0]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, bidirectional=bidirectional, dropout=dropout)
        self.fc = nn.Linear(hidden_dim*2, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        #x = [sent len, batch size]
        #print(x)
        
        embedded = self.dropout(self.embedding(x))
        ##print("embedded.size()", embedded.size())
        
        #embedded = [sent len, batch size, emb dim]
        
        output, (hidden, cell) = self.rnn(embedded)
        ##print("hidden1.size()", hidden.size())
        
        #output = [sent len, batch size, hid dim * num directions]
        #hidden = [num layers * num directions, batch size, hid. dim]
        #cell = [num layers * num directions, batch size, hid. dim]
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
        ##print("hidden2.size()", hidden.size())
                
        #hidden [batch size, hid. dim * num directions]
        out = self.fc(hidden.squeeze(0))
        ##print("out.size()", out.size())

        return out

Criaremos uma instância da classe RNN com os novos parâmetros e argumentos para o número de camadas, bidirecionalidade e probabilidade de desistência.

Neste exemplo, usamos o embedding padrão do pytorch, sem pré-treinamento

In [0]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 50 #100
HIDDEN_DIM = 64 #256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, N_LAYERS, BIDIRECTIONAL, DROPOUT)

### Treinado o Modelo

O otimizador de `SGD` a ser usado será o ` Adam`. O SGD atualiza todos os parâmetros com a mesma taxa de aprendizado e a escolha dessa taxa de aprendizado pode ser complicada. O Adam adapta a taxa de aprendizado para cada parâmetro, fornecendo taxas de aprendizado mais baixas a parâmetros atualizados com mais frequência, e taxas de aprendizado mais altas a parâmetros atualizados com frequência mais baixa. Mais informações sobre Adam (e outros otimizadores) podem ser encontradas [aqui](http://ruder.io/optimizing-gradient-descent/index.html).

Para alterar `SGD` para` Adam`, simplesmente alteramos `optim.SGD` para` optim.Adam`, observe também como não precisamos fornecer uma taxa de aprendizado inicial para Adam, pois o PyTorch especifica uma taxa de aprendizado inicial sensata.

In [0]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

Definimos o critério e colocamos o modelo e o critério na GPU (se disponível).

In [0]:
criterion = nn.BCEWithLogitsLoss()
model = model.to(device)
criterion = criterion.to(device)

Implementamos a função para calcular a precisão.

In [0]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum()/len(correct)
    return acc

Definimos uma função para treinar nosso modelo.

**Nota**: como agora estamos usando o dropout, devemos lembrar de usar `model.train ()` para garantir que o dropout seja "ativado" durante o treinamento.

In [0]:
def train(model, iterator, optimizer, criterion):
    print("Treinando ...") ##
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    i=0
    
    for batch in iterator:
        #print("Texto", batch.Texto)
        optimizer.zero_grad()
        i += 1
        
        if len(batch.Texto[0]) < 3:  ## debug
            print("batch", i, batch.Texto[0])
            continue
        
        ##print("Texto", batch.Texto[0])
        src = torch.transpose(batch.Texto[0],0,1)
        predictions = model(src)
        ##print(predictions.size())
        predictions=predictions.squeeze(1)
        ##print(predictions.size())
        
        ##print("Julgamento", batch.Julgamento)
        loss = criterion(predictions, batch.Julgamento)
        
        acc = binary_accuracy(predictions, batch.Julgamento)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

Definimos uma função para testar nosso modelo.

**Nota**: como agora estamos usando o dropout, devemos lembrar de usar `model.eval ()` para garantir que o dropout seja "desligado" durante a avaliação.

In [0]:
def evaluate(model, iterator, optimizer, criterion):
    print("Avaliando ...") ##
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    i=0
    with torch.no_grad():
    
        for batch in iterator:
            i+=1
            
            if len(batch.Texto[0]) < 3:  ## debug
                print("batch", i, batch.Texto[0])
                continue

            src = torch.transpose(batch.Texto[0],0,1)
        
            predictions = model(src).squeeze(1)
            
            loss = criterion(predictions, batch.Julgamento)
            
            acc = binary_accuracy(predictions, batch.Julgamento)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

Finalmente, treinamos nosso modelo.

In [0]:
N_EPOCHS = 5

for epoch in range(N_EPOCHS):
    print("Starting Epoch:", epoch+1)
  
    train_loss, train_acc = train(model, trainIter, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, validIter, optimizer, criterion)
    
    print(f'Epoch: {epoch+1:02}, Train Loss: {train_loss:.3f}, Train Acc: {train_acc*100:.2f}%, Val. Loss: {valid_loss:3f}, Val. Acc: {valid_acc*100:.2f}%')

Starting Epoch: 1
Treinando ...
batch 1001 tensor([[  2,  13,  19,  10,   4, 138,   8,   6, 412,  12,  34,  81,  28,  21,
           4,   3]], device='cuda:0')
Avaliando ...
Epoch: 01, Train Loss: 0.427, Train Acc: 79.62%, Val. Loss: 0.341282, Val. Acc: 84.46%
Starting Epoch: 2
Treinando ...
batch 1001 tensor([[  2, 107, 370,   6,  12,  36, 267,   5, 175, 207,   5,   7,  14,  33,
          92, 133,   4,   3]], device='cuda:0')
Avaliando ...
Epoch: 02, Train Loss: 0.340, Train Acc: 85.18%, Val. Loss: 0.299502, Val. Acc: 87.36%
Starting Epoch: 3
Treinando ...
batch 1001 tensor([[  2, 115,  40,   0,   5,  14,  18,  10,  13,  19,   5,   7, 115, 656,
          17,  35, 406,   3]], device='cuda:0')
Avaliando ...
Epoch: 03, Train Loss: 0.308, Train Acc: 86.63%, Val. Loss: 0.277161, Val. Acc: 88.77%
Starting Epoch: 4
Treinando ...
batch 1001 tensor([[  2, 747,  18, 315, 398,   0, 107, 524,  15, 330,  29,  59, 206,   4,
          10,   0,   4,   3]], device='cuda:0')
Avaliando ...
Epoch: 04, Tr

Testamos a acurácia em dados nunca vistos durante o treinamento.

In [0]:
test_loss, test_acc = evaluate(model, testIter, optimizer, criterion)

print(f'Test Loss: {test_loss:.3f}, Test Acc: {test_acc*100:.2f}%')

Avaliando ...
Test Loss: 0.238, Test Acc: 89.97%


### Entrada do usuário

Agora podemos usar nosso modelo para prever o sentimento de qualquer sentença que damos a ele. Como foi treinado em resenhas de produtos, as frases fornecidas também devem ser resenhas de produtos.

Nossa função `predict_sentiment` faz algumas coisas:
- tokeniza a frase, ou seja, divide-a de uma sequência bruta em uma lista de tokens
- indexa os tokens convertendo-os em sua representação inteira do nosso vocabulário
- converte os índices, que são uma lista Python em um tensor PyTorch
- adicione uma dimensão de lote ao cancelar a seleção
- esmaga a previsão de saída de um número real entre 0 e 1 com a função `sigmoid`
- converte o tensor segurando um único valor em um inteiro com o método `item ()`

Esperamos que revisões com um sentimento negativo retornem um valor próximo de 0 e revisões positivas que retornem um valor próximo de 1.

In [0]:
def predict_sentiment(sentence):
    tokenized = tokenizePT(sentence)
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    pred = model(tensor)
    prediction = torch.sigmoid(pred)
    return prediction.item()

print(predict_sentiment("Essa droga não serve para nada")) 

print(predict_sentiment("Esse produto é maravilhoso!!!"))

0.2131531834602356
0.9384145736694336
